In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from functools import reduce

In [2]:
data = pd.read_csv('zipcode_data.csv')
covid = pd.read_csv('data-by-modzcta.csv')

C:\Users\Earl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,38,39,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,58,59,62,63,64,66,67,70,71,72,74,75,78,79,82,83,86,87,90,91,94,95,102,103,104,106,107,110,111,114,115,118,119,122,123,126,127,128,130,131,134,135,138,139,142,143,146,147,150,151,154,155,158,159,162,163,166,167,170,171,174,175,178,179,182,183,184,186,187,190,191,194,195,198,199,202,203,204,206,207,210,211,214,215,218,219,222,223,226,227,230,231,234,235,238,239,242,243,254,255,262,263,270,271,278,279,286,287,294,295,298,299,300,302,303,306,307,310,311,314,315,318,319,322,323,326,327,330,331,334,335,338,339,354,355,356,384,385,388,389,392,393,396,397,406,407,408,410,411,412,414,415,416,418,419,422,423,426,427,430,431,434,435,436,438,439,442,443,446,447,450,451,454,455,456,458,459,462,463,466,467,470,471,512,513,532,533) have mixed types. Specify dtype 

In [3]:
covid.shape

(177, 10)

In [4]:
# zipcodes = ['11201','11203','11204','11205','11206','11207','11208','11209','11210','11211','11212','11213','11214',
#             '11215','11216','11217','11218','11219','11220','11221','11222','11223','11224','11225','11226','11228',
#             '11229','11230','11231','11232','11233','11234','11235','11236','11237','11238','11239','11249','11251']

In [5]:
data.head()

,id,Geographic Area Name,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over,Percent Estimate!!EMPLOYMENT STATUS!!Population 16 years and over,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Percent Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,...,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!18 years and over!!65 years and over,Percent Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!18 years and over!!65 years and over,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!People in families,Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!People in families,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!People in families,Percent Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!People in families,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Unrelated individuals 15 years and over,Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Unrelated individuals 15 years and over,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Unrelated individuals 15 years and over,Percent Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Unrelated individuals 15 years and over
0,8600000US00601,ZCTA5 00601,13924,280,13924,(X),5811,454,41.7,3.2,...,59.9,5.8,(X),(X),61.9,5.4,(X),(X),68.3,5.7
1,8600000US00602,ZCTA5 00602,32020,147,32020,(X),13493,683,42.1,2.1,...,44.7,5.3,(X),(X),49.1,3.6,(X),(X),64.1,5.2
2,8600000US00603,ZCTA5 00603,40180,663,40180,(X),15595,753,38.8,1.8,...,41.3,3.3,(X),(X),48.8,2.8,(X),(X),63.9,3.6
3,8600000US00606,ZCTA5 00606,5365,279,5365,(X),1585,218,29.5,3.7,...,40.7,10.5,(X),(X),63.1,7.3,(X),(X),73.3,8.8
4,8600000US00610,ZCTA5 00610,22641,221,22641,(X),9754,590,43.1,2.6,...,39.3,5.1,(X),(X),41.7,3.8,(X),(X),57,7


#### Transporation and Occupation

In [6]:
covid = covid.rename(columns={"MODIFIED_ZCTA": "zipcode"})

In [7]:
covid['zipcode'] = covid['zipcode'].astype(str)

In [8]:
s = data['Geographic Area Name'].str.split(' ', expand=True)

In [9]:
data['zipcode'] = s[1]

In [10]:
feature_list=['zipcode',
'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- drove alone',
'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- carpooled',
'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)',
'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Walked',
'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Other means',
'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Worked at home',
'Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)',
'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Management, business, science, and arts occupations',
'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Service occupations',
'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Sales and office occupations',
'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Natural resources, construction, and maintenance occupations',
'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Production, transportation, and material moving occupations',
'Percent Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!No health insurance coverage',
'Estimate!!INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)'
]

In [11]:
data2 = data[feature_list]

In [12]:
data3 = data2.merge(covid, on='zipcode')

In [13]:
data3.shape

(177, 24)

In [68]:
data3 = data3.rename(columns={'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- drove alone': 'pct_drove_alone',
                             'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- carpooled': 'pct_carpooled',
                             'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)':'pct_public_transporation',
                             'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Walked':'pct_walked',
                             'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Other means': 'pct_other_transportation',
                             'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Worked at home':'pct_worked_at_home',
                             'Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)': 'mean_travel_time',
                             'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Management, business, science, and arts occupations': 'Management_business_science_arts_occupations',
                             'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Service occupations':'Service_occupations',
                             'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Sales and office occupations':'Sales_office_occupations',
                             'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Natural resources, construction, and maintenance occupations':'Natural_resources_construction_maintenance_occupations',
                             'Percent Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Production, transportation, and material moving occupations': 'Production_transportation_material_moving_occupations',
                             'Percent Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!No health insurance coverage':'pct_no_health_insurance',
                             'Estimate!!INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)':'median_household_income'})

In [15]:
ols_features = [
    'pct_drove_alone',
    #'pct_carpooled', not siginifcant
    'pct_public_transporation',
    'pct_walked',
    #'pct_other_transportation', not significant
    'pct_worked_at_home'

]

In [16]:
data3[ols_features] = data3[ols_features].astype(float)

In [17]:
ols_str = "+".join(ols_features) + '-1'
#ols_str = "+".join(ols_features)

In [18]:
ols_str

'pct_drove_alone+pct_public_transporation+pct_walked+pct_worked_at_home-1'

In [19]:
lm = smf.ols(formula='PERCENT_POSITIVE ~ ' + ols_str, data = data3).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:       PERCENT_POSITIVE   R-squared (uncentered):                   0.967
Model:                            OLS   Adj. R-squared (uncentered):              0.966
Method:                 Least Squares   F-statistic:                              1250.
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                   1.87e-126
Time:                        13:03:02   Log-Likelihood:                         -468.63
No. Observations:                 177   AIC:                                      945.3
Df Residuals:                     173   BIC:                                      958.0
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

In [20]:
lm = smf.ols(formula='COVID_CASE_RATE ~ ' + ols_str, data = data3).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:        COVID_CASE_RATE   R-squared (uncentered):                   0.936
Model:                            OLS   Adj. R-squared (uncentered):              0.935
Method:                 Least Squares   F-statistic:                              633.1
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                   4.16e-102
Time:                        13:03:02   Log-Likelihood:                         -1394.5
No. Observations:                 177   AIC:                                      2797.
Df Residuals:                     173   BIC:                                      2810.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

In [21]:
lm = smf.ols(formula='COVID_DEATH_RATE ~ ' + ols_str, data = data3).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:       COVID_DEATH_RATE   R-squared (uncentered):                   0.841
Model:                            OLS   Adj. R-squared (uncentered):              0.837
Method:                 Least Squares   F-statistic:                              227.9
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                    8.01e-68
Time:                        13:03:02   Log-Likelihood:                         -1053.8
No. Observations:                 177   AIC:                                      2116.
Df Residuals:                     173   BIC:                                      2128.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

In [22]:
ols_str1 = ols_str + '-1'

# get y and X dataframes based on this regression:
y, X = dmatrices('COVID_CASE_RATE ~' + ols_str1, data3, return_type='dataframe')

# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

,VIF Factor,features
0,1.842385,pct_drove_alone
1,5.241997,pct_public_transporation
2,3.516155,pct_walked
3,6.904590,pct_worked_at_home


In [23]:
data3.head()

,zipcode,pct_drove_alone,pct_carpooled,pct_public_transporation,pct_walked,pct_other_transportation,pct_worked_at_home,mean_travel_time,Management_business_science_arts_occupations,Service_occupations,...,median_household_income,NEIGHBORHOOD_NAME,BOROUGH_GROUP,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS
0,10001,3.0,1,49.7,32.2,7.6,6.4,27.9,64.6,10.4,...,88526,Chelsea/NoMad/West Chelsea,Manhattan,387,1642.40,23563.03,23,97.61,10.25,3777
1,10002,5.1,3.6,54.6,25.6,7,4.2,32.7,44.5,25.4,...,35859,Chinatown/Lower East Side,Manhattan,1096,1427.91,76755.41,158,205.85,14.30,7667
2,10003,4.4,1.5,51.1,27.1,8.2,7.7,25.2,71.2,8,...,112131,East Village/Gramercy/Greenwich Village,Manhattan,471,875.44,53801.62,33,61.34,7.93,5936
3,10004,7.5,1,54.3,24.4,1.1,11.8,25.4,76.2,2.4,...,157645,Financial District,Manhattan,32,876.57,3650.61,1,27.39,8.21,390
4,10005,1.6,0.2,62.8,29.9,1.1,4.4,24.3,71.3,3.2,...,173333,Financial District,Manhattan,65,774.17,8396.11,2,23.82,7.39,880


#### Occupation

In [57]:
ols_features = [
    'Service_occupations',
    'Sales_office_occupations',
    'Natural_resources_construction_maintenance_occupations',
    'Production_transportation_material_moving_occupations'
]

In [60]:
data3[ols_features] = data3[ols_features].astype(float)

In [61]:
ols_str = "+".join(ols_features) + '-1'

In [62]:
lm = smf.ols(formula='PERCENT_POSITIVE ~ ' + ols_str, data = data3).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:       PERCENT_POSITIVE   R-squared (uncentered):                   0.979
Model:                            OLS   Adj. R-squared (uncentered):              0.978
Method:                 Least Squares   F-statistic:                              2008.
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                   9.11e-144
Time:                        13:08:19   Log-Likelihood:                         -427.84
No. Observations:                 177   AIC:                                      863.7
Df Residuals:                     173   BIC:                                      876.4
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                                                             coef    std err          t      P>|t|      [0.025      0.97

In [63]:
lm = smf.ols(formula='COVID_CASE_RATE ~ ' + ols_str, data = data3).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:        COVID_CASE_RATE   R-squared (uncentered):                   0.946
Model:                            OLS   Adj. R-squared (uncentered):              0.944
Method:                 Least Squares   F-statistic:                              753.8
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                   2.83e-108
Time:                        13:08:39   Log-Likelihood:                         -1379.9
No. Observations:                 177   AIC:                                      2768.
Df Residuals:                     173   BIC:                                      2781.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                                                             coef    std err          t      P>|t|      [0.025      0.97

In [64]:
lm = smf.ols(formula='COVID_DEATH_RATE ~ ' + ols_str, data = data3).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:       COVID_DEATH_RATE   R-squared (uncentered):                   0.856
Model:                            OLS   Adj. R-squared (uncentered):              0.853
Method:                 Least Squares   F-statistic:                              257.9
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                    9.47e-72
Time:                        13:08:51   Log-Likelihood:                         -1044.6
No. Observations:                 177   AIC:                                      2097.
Df Residuals:                     173   BIC:                                      2110.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                                                             coef    std err          t      P>|t|      [0.025      0.97

#### Demographic

In [24]:
demographic = pd.read_csv('Demographic_Statistics_By_Zip_Code.csv')

In [25]:
demographic = demographic.rename(columns={'JURISDICTION NAME':'zipcode'})
demographic['zipcode'] = demographic['zipcode'].astype(str)

In [26]:
covid_demo = covid.merge(demographic, on='zipcode')

In [27]:
demographic.columns

Index(['zipcode', 'COUNT PARTICIPANTS', 'COUNT FEMALE', 'PERCENT FEMALE',
       'COUNT MALE', 'PERCENT MALE', 'COUNT GENDER UNKNOWN',
       'PERCENT GENDER UNKNOWN', 'COUNT GENDER TOTAL', 'PERCENT GENDER TOTAL',
       'COUNT PACIFIC ISLANDER', 'PERCENT PACIFIC ISLANDER',
       'COUNT HISPANIC LATINO', 'PERCENT HISPANIC LATINO',
       'COUNT AMERICAN INDIAN', 'PERCENT AMERICAN INDIAN',
       'COUNT ASIAN NON HISPANIC', 'PERCENT ASIAN NON HISPANIC',
       'COUNT WHITE NON HISPANIC', 'PERCENT WHITE NON HISPANIC',
       'COUNT BLACK NON HISPANIC', 'PERCENT BLACK NON HISPANIC',
       'COUNT OTHER ETHNICITY', 'PERCENT OTHER ETHNICITY',
       'COUNT ETHNICITY UNKNOWN', 'PERCENT ETHNICITY UNKNOWN',
       'COUNT ETHNICITY TOTAL', 'PERCENT ETHNICITY TOTAL',
       'COUNT PERMANENT RESIDENT ALIEN', 'PERCENT PERMANENT RESIDENT ALIEN',
       'COUNT US CITIZEN', 'PERCENT US CITIZEN', 'COUNT OTHER CITIZEN STATUS',
       'PERCENT OTHER CITIZEN STATUS', 'COUNT CITIZEN STATUS UNKNOWN',
    

In [28]:
covid_demo = covid_demo.rename(columns={'PERCENT PACIFIC ISLANDER':'pct_pacific_islander',
    'PERCENT HISPANIC LATINO':'pct_hispanic',
    'PERCENT AMERICAN INDIAN':'pct_american_indian',
    'PERCENT ASIAN NON HISPANIC':'pct_asian',
    'PERCENT WHITE NON HISPANIC':'pct_white',
    'PERCENT BLACK NON HISPANIC':'pct_black',
    'PERCENT OTHER ETHNICITY':'pct_other'})

In [29]:
ols_features =[
    'pct_pacific_islander',
    'pct_hispanic',
    'pct_american_indian',
    'pct_asian',
    'pct_white',
    'pct_black',
    'pct_other'
]

In [30]:
ols_str = "+".join(ols_features) + '-1'

In [31]:
lm = smf.ols(formula='PERCENT_POSITIVE ~ ' + ols_str, data = covid_demo).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:       PERCENT_POSITIVE   R-squared (uncentered):                   0.447
Model:                            OLS   Adj. R-squared (uncentered):              0.423
Method:                 Least Squares   F-statistic:                              18.57
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                    5.16e-18
Time:                        13:03:02   Log-Likelihood:                         -682.36
No. Observations:                 168   AIC:                                      1379.
Df Residuals:                     161   BIC:                                      1401.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [32]:
lm = smf.ols(formula='COVID_CASE_RATE ~ ' + ols_str, data = covid_demo).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:        COVID_CASE_RATE   R-squared (uncentered):                   0.471
Model:                            OLS   Adj. R-squared (uncentered):              0.449
Method:                 Least Squares   F-statistic:                              20.52
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                    1.48e-19
Time:                        13:03:02   Log-Likelihood:                         -1501.9
No. Observations:                 168   AIC:                                      3018.
Df Residuals:                     161   BIC:                                      3040.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [33]:
lm = smf.ols(formula='COVID_DEATH_RATE ~ ' + ols_str, data = covid_demo).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:       COVID_DEATH_RATE   R-squared (uncentered):                   0.459
Model:                            OLS   Adj. R-squared (uncentered):              0.436
Method:                 Least Squares   F-statistic:                              19.53
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                    8.82e-19
Time:                        13:03:02   Log-Likelihood:                         -1104.7
No. Observations:                 168   AIC:                                      2223.
Df Residuals:                     161   BIC:                                      2245.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

#### Brooklyn Environmental

In [34]:
medical = pd.read_csv('facility per person.csv')
medical['ZIPCODE'] = medical['ZIPCODE'].astype(str)
medical = medical.rename(columns={'ZIPCODE':'zipcode','facilities per 100,000 person':'facilities_per_100000','POPULATION':'population_na'})

In [35]:
parks = pd.read_csv('park.csv')
parks['ZIPCODE'] = parks['ZIPCODE'].astype(str)
parks = parks.rename(columns={'ZIPCODE':'zipcode','park/sqft':'park_sqft','park/pop':'park_pop'})

In [36]:
air = pd.read_csv('air_pollution.csv')
air['ZIPCODE'] = air['ZIPCODE'].astype(str)
air = air.rename(columns={'ZIPCODE':'zipcode'})

In [39]:
trees = pd.read_csv('tree_count_calc.csv')
trees['ZIPCODE'] = trees['ZIPCODE'].astype(str)
trees = trees.rename(columns={'ZIPCODE':'zipcode','tree/sqft':'tree_sqft'})
trees['tree_1000_sqft'] = trees['tree_sqft']*1000

In [42]:
call_311 = pd.read_csv('brooklyn_311_covid.csv')
call_311['zipcode'] = call_311['zipcode'].astype(str)

In [90]:
income = data3[['median_household_income','zipcode']]

In [92]:
data_frames = [covid, parks, air, trees, medical, call_311,income]

In [93]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['zipcode'],how='inner'), data_frames)

In [94]:
df_merged['area_sqmi'] = df_merged['AREA'] / (5280*5280)

In [95]:
df_merged['pop_density'] = df_merged['POPULATION'] / df_merged['area_sqmi']

In [96]:
df_merged['tree_per_sqmi'] = df_merged['tree_count'] / df_merged['area_sqmi']

In [97]:
df_merged['park_sqmi'] = df_merged['shape_ar_1'] / (5280*5280)

In [98]:
df_merged['park_per_person'] = df_merged['park_sqmi'] / df_merged['POPULATION']

In [76]:
df_merged['park_per_area'] = df_merged['park_sqmi'] / df_merged['area_sqmi']

In [112]:
df_merged['median_household_income'] = df_merged['median_household_income'].astype(float)

In [113]:
ols_features =[
    'park_sqmi',
    'park_per_person',
    '_mean', 
    'tree_count',
    'tree_per_sqmi',
    'pop_density', #people/sqmi
    'facilities_per_100000',
    'count_2020',
    'median_household_income'
]

In [114]:
ols_str = "+".join(ols_features) + '-1'

In [115]:
lm = smf.ols(formula='PERCENT_POSITIVE ~ ' + ols_str, data = df_merged).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:       PERCENT_POSITIVE   R-squared (uncentered):                   0.975
Model:                            OLS   Adj. R-squared (uncentered):              0.968
Method:                 Least Squares   F-statistic:                              150.6
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                    2.80e-25
Time:                        13:17:33   Log-Likelihood:                         -105.22
No. Observations:                  44   AIC:                                      228.4
Df Residuals:                      35   BIC:                                      244.5
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

In [117]:
lm = smf.ols(formula='COVID_CASE_RATE ~ ' + ols_str, data = df_merged).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:        COVID_CASE_RATE   R-squared (uncentered):                   0.965
Model:                            OLS   Adj. R-squared (uncentered):              0.956
Method:                 Least Squares   F-statistic:                              106.9
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                    9.06e-23
Time:                        13:17:45   Log-Likelihood:                         -325.49
No. Observations:                  44   AIC:                                      669.0
Df Residuals:                      35   BIC:                                      685.0
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

In [118]:
lm = smf.ols(formula='COVID_DEATH_RATE ~ ' + ols_str, data = df_merged).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:       COVID_DEATH_RATE   R-squared (uncentered):                   0.918
Model:                            OLS   Adj. R-squared (uncentered):              0.896
Method:                 Least Squares   F-statistic:                              43.28
Date:                Wed, 15 Jul 2020   Prob (F-statistic):                    2.38e-16
Time:                        13:17:47   Log-Likelihood:                         -246.43
No. Observations:                  44   AIC:                                      510.9
Df Residuals:                      35   BIC:                                      526.9
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------